In [1]:
%pip install redis

Note: you may need to restart the kernel to use updated packages.


In [20]:
import redis

r = redis.Redis(host='localhost', port=6379, decode_responses=True, db=0)

ping = r.ping()
print("Connected to Redis!" if ping else "Connection failed.")



Connected to Redis!


## Q1: How many actors and movies are stored in Redis?

In [21]:
actor_keys = list(r.scan_iter("actor:*"))
movie_keys = list(r.scan_iter("movie:*"))

print("Total actors:", len(actor_keys))
print("Total movies:", len(movie_keys))


Total actors: 1319
Total movies: 922


## Q2: List 5 actors born before 1980

In [22]:
actors_pre_1980 = []

for key in actor_keys:
    dob = r.hget(key, "date_of_birth")
    if dob and int(dob) < 1980:
        actor = r.hgetall(key)
        actors_pre_1980.append(actor)
    if len(actors_pre_1980) >= 5:
        break

for a in actors_pre_1980:
    print(f"{a['first_name']} {a['last_name']} - {a['date_of_birth']}")


Vincent Price - 1911
Dolph Lundgren - 1957
Lance Reddick - 1962
Jessica Lange - 1949
Adam Scott - 1973


## Q3: Genre and rating of "The Imitation Game"

In [23]:
for key in movie_keys:
    title = r.hget(key, "title")
    if title == "The Imitation Game":
        genre = r.hget(key, "genre")
        rating = r.hget(key, "rating")
        print(f"Genre: {genre}, Rating: {rating}")
        break


Genre: Biography, Rating: 8.1


## Q4: Top 5 highest-rated movies

In [24]:
movies = []

for key in movie_keys:
    try:
        rating = float(r.hget(key, "rating"))
        title = r.hget(key, "title")
        movies.append((title, rating))
    except:
        continue

top_5 = sorted(movies, key=lambda x: x[1], reverse=True)[:5]

for title, rating in top_5:
    print(f"{title}: {rating}")


Boy 9: 9.4
Vegas (doc): 9.4
The Shawshank Redemption: 9.3
Ween Live in Chicago: 9.2
Over Canada: An Aerial Adventure: 9.1


## Q5: How many movies have rating above 7.5

In [25]:
count = 0

for key in movie_keys:
    try:
        rating = float(r.hget(key, "rating"))
        if rating > 7.5:
            count += 1
    except:
        continue

print("Movies with rating > 7.5:", count)


Movies with rating > 7.5: 183


## Q6: Update "The Imitation Game" to rating 8.5

In [26]:
for key in movie_keys:
    title = r.hget(key, "title")
    if title == "The Imitation Game":
        r.hset(key, "rating", 8.5)
        print(f"Updated {title} to rating 8.5")
        break


Updated The Imitation Game to rating 8.5


## Q8: Delete movie titled "The Room"

In [27]:
for key in movie_keys:
    title = r.hget(key, "title")
    if title == "The Room":
        r.delete(key)
        print("Deleted movie:", title)
        break
